# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[28020, 28021, 28022]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [8]:
fcst.fit()

MODEL DOMAIN
28020 Will the Boeing Starliner Commercial Crew program be canceled before October 1, 2024?
Aerospace technology
28021 Will the New York Yankees win more games than the Baltimore Orioles in the 2024 MLB season, before October 1, 2024?
Sports performance
28022 Will SpaceX make any new posts on its Updates page before October 1, 2024? 
Aerospace technology
CORRELATOR
|Boeing Starliner Program|28020|Will the Boeing Starliner Commercial Crew program be canceled before October 1, 2024?|
|2024 MLB Season Outcome|28021|Will the New York Yankees win more games than the Baltimore Orioles in the 2024 MLB season, before October 1, 2024?|
|SpaceX Updates|28022|Will SpaceX make any new posts on its Updates page before October 1, 2024?|
NEWS
ASKING
Researching Will the Boeing Starliner Commercial Crew program be canceled before October 1, 2024?
ALL
NASA's decision to leave astronauts Butch Wilmore and Suni Williams on the International Space Station due to issues with Boeing's Starliner

In [9]:
for ifp in ifps.values():
    try:
        print(ifp.id, ifp.forecast)
    except:
        print(ifp.id, 'none')

28020 10
28021 60
28022 85


In [10]:
fcst.report()

28020 Will the Boeing Starliner Commercial Crew program be canceled before October 1, 2024?
Forecast 10
Rationale Given the current timeline and indications from both Boeing and NASA, it is unlikely that the Boeing Starliner Commercial Crew program will be canceled before October 1, 2024. While there are severe technical issues and financial concerns, both parties appear committed to continue working on resolving these issues and have not indicated any immediate plans to cease the program. Therefore, the probability of cancellation by the specified date is low. 

28021 Will the New York Yankees win more games than the Baltimore Orioles in the 2024 MLB season, before October 1, 2024?
Forecast 60
Rationale Given the slight advantages in schedule difficulty and home games for the Yankees, along with their recent performance improvements and the Orioles' more challenging remaining schedule, I estimate a 60% probability that the Yankees will finish the 2024 MLB season with more wins than th

In [11]:
fcst.upload()

Prediction posted for  28020
Comment posted for  28020
Prediction posted for  28021
Comment posted for  28021
Prediction posted for  28022
Comment posted for  28022
